In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/loan-approval-prediction-dataset/loan_approval_dataset.csv


In [2]:
import torch
from sklearn.preprocessing import MinMaxScaler ,LabelEncoder

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [3]:
df = pd.read_csv('/kaggle/input/loan-approval-prediction-dataset/loan_approval_dataset.csv',index_col = 'loan_id')
df

,no_of_dependents,education,self_employed,income_annum,loan_amount,loan_term,cibil_score,residential_assets_value,commercial_assets_value,luxury_assets_value,bank_asset_value,loan_status
loan_id,,,,,,,,,,,,
1,2,Graduate,No,9600000,29900000,12,778,2400000,17600000,22700000,8000000,Approved
2,0,Not Graduate,Yes,4100000,12200000,8,417,2700000,2200000,8800000,3300000,Rejected
3,3,Graduate,No,9100000,29700000,20,506,7100000,4500000,33300000,12800000,Rejected
4,3,Graduate,No,8200000,30700000,8,467,18200000,3300000,23300000,7900000,Rejected
5,5,Not Graduate,Yes,9800000,24200000,20,382,12400000,8200000,29400000,5000000,Rejected
...,...,...,...,...,...,...,...,...,...,...,...,...
4265,5,Graduate,Yes,1000000,2300000,12,317,2800000,500000,3300000,800000,Rejected
4266,0,Not Graduate,Yes,3300000,11300000,20,559,4200000,2900000,11000000,1900000,Approved
4267,2,Not Graduate,No,6500000,23900000,18,457,1200000,12400000,18100000,7300000,Rejected


In [4]:
df[df.isna() == True].count()

 no_of_dependents            0
 education                   0
 self_employed               0
 income_annum                0
 loan_amount                 0
 loan_term                   0
 cibil_score                 0
 residential_assets_value    0
 commercial_assets_value     0
 luxury_assets_value         0
 bank_asset_value            0
 loan_status                 0
dtype: int64

# Note here there is a problem in dataset column as there is space in names which cause problem so let us rename the columns.

In [5]:
df.rename(columns = {' no_of_dependents': 'no_of_dependents',
                     ' education' : 'education', 
                     ' self_employed' : 'self_employed',
                     ' income_annum' : 'income_annum',
                     ' loan_amount' : 'loan_amount',
                     ' loan_term' :'loan_term',
                     ' cibil_score' : 'cibil_score',
                     ' residential_assets_value' : 'residential_assets_value', 
                     ' commercial_assets_value' : 'commercial_assets_value',
                     ' luxury_assets_value' : 'luxury_assets_value',
                     ' bank_asset_value' : 'bank_asset_value',
                     ' loan_status' : 'loan_status'}, 
                      inplace = True)
df.columns
col = ['no_of_dependents', 'income_annum', 'loan_amount', 'loan_term',
       'cibil_score', 'residential_assets_value', 'commercial_assets_value',
       'luxury_assets_value', 'bank_asset_value', 'education_ Graduate',
       'education_ Not Graduate', 'self_employed_ No', 'self_employed_ Yes',
       'Total_assets_value']
Scaling = ['residential_assets_value','commercial_assets_value',	'luxury_assets_value'	,'bank_asset_value']

In [6]:
label_encoder  =LabelEncoder()
mx = MinMaxScaler()
def preprocess(df):
    y = df.pop('loan_status')
    y = label_encoder.fit_transform(y)
    
    df = pd.get_dummies(df)
    
    df['Total_assets_value'] = 0
    for i in Scaling:
        df['Total_assets_value'] += df[i] 
    
    df[col]=mx.fit_transform(df[col])
    df.drop(columns = Scaling)
    
    
    return df,y

df,y = preprocess(df)

In [7]:
df

,no_of_dependents,income_annum,loan_amount,loan_term,cibil_score,residential_assets_value,commercial_assets_value,luxury_assets_value,bank_asset_value,education_ Graduate,education_ Not Graduate,self_employed_ No,self_employed_ Yes,Total_assets_value
loan_id,,,,,,,,,,,,,,
1,0.4,0.969072,0.755102,0.555556,0.796667,0.085616,0.907216,0.575835,0.544218,1.0,0.0,1.0,0.0,0.557032
2,0.0,0.402062,0.303571,0.333333,0.195000,0.095890,0.113402,0.218509,0.224490,0.0,1.0,0.0,1.0,0.183832
3,0.6,0.917526,0.750000,1.000000,0.343333,0.246575,0.231959,0.848329,0.870748,1.0,0.0,1.0,0.0,0.634551
4,0.6,0.824742,0.775510,0.333333,0.278333,0.626712,0.170103,0.591260,0.537415,1.0,0.0,1.0,0.0,0.579181
5,1.0,0.989691,0.609694,1.000000,0.136667,0.428082,0.422680,0.748072,0.340136,0.0,1.0,0.0,1.0,0.604651
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4265,1.0,0.082474,0.051020,0.555556,0.028333,0.099315,0.025773,0.077121,0.054422,1.0,0.0,0.0,1.0,0.077519
4266,0.0,0.319588,0.280612,1.000000,0.431667,0.147260,0.149485,0.275064,0.129252,0.0,1.0,0.0,1.0,0.217054
4267,0.4,0.649485,0.602041,0.888889,0.261667,0.044521,0.639175,0.457584,0.496599,0.0,1.0,1.0,0.0,0.427464


In [8]:
y

array([0, 1, 1, ..., 1, 0, 0])